In [6]:
#Final Version

import pygame
import random

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

# Define screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Define player attributes
PLAYER_SIZE = 50
PLAYER_SPEED = 5

# Define enemy attributes
ENEMY_SIZE = 30
ENEMY_SPEED = 3

# Define game attributes
SCORE_INCREMENT = 10

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([PLAYER_SIZE, PLAYER_SIZE])
        self.image.fill(GREEN)
        self.rect = self.image.get_rect()
        self.score = 0
        self.health = 100

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and self.rect.x > 0:
            self.rect.x -= PLAYER_SPEED
        if keys[pygame.K_RIGHT] and self.rect.x < SCREEN_WIDTH - PLAYER_SIZE:
            self.rect.x += PLAYER_SPEED
        if keys[pygame.K_UP] and self.rect.y > 0:
            self.rect.y -= PLAYER_SPEED
        if keys[pygame.K_DOWN] and self.rect.y < SCREEN_HEIGHT - PLAYER_SIZE:
            self.rect.y += PLAYER_SPEED

    def increase_score(self):
        self.score += SCORE_INCREMENT

    def decrease_health(self, amount):
        self.health -= amount

class Enemy(pygame.sprite.Sprite):
    def __init__(self, player):
        super().__init__()
        self.image = pygame.Surface([ENEMY_SIZE, ENEMY_SIZE])
        self.image.fill(RED)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, SCREEN_WIDTH - ENEMY_SIZE)
        self.rect.y = random.randint(0, SCREEN_HEIGHT - ENEMY_SIZE)
        self.player = player

    def update(self):
        vector_x = self.player.rect.x - self.rect.x
        vector_y = self.player.rect.y - self.rect.y
        distance = max(1, pygame.math.Vector2(vector_x, vector_y).length())
        direction_x = vector_x / distance
        direction_y = vector_y / distance
        self.rect.x += direction_x * ENEMY_SPEED
        self.rect.y += direction_y * ENEMY_SPEED

class Wall(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height):
        super().__init__()
        self.image = pygame.Surface([width, height])
        self.image.fill(BLACK)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([20, 20])
        self.image.fill(YELLOW)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, SCREEN_WIDTH - 20)
        self.rect.y = random.randint(0, SCREEN_HEIGHT - 20)

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Object-Oriented Game")
        self.clock = pygame.time.Clock()
        self.all_sprites = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()
        self.walls = pygame.sprite.Group()
        self.player = Player()
        self.all_sprites.add(self.player)
        self.generate_enemies()
        self.generate_walls()
        self.generate_obstacles()
        self.coins = pygame.sprite.Group()
        self.generate_coins()


    def generate_enemies(self):
        for _ in range(5):
            enemy = Enemy(self.player)
            self.enemies.add(enemy)
            self.all_sprites.add(enemy)

    def generate_walls(self):
        # Top and bottom walls
        for x in range(0, SCREEN_WIDTH, 50):
            wall_top = Wall(x, 0, 50, 10)
            wall_bottom = Wall(x, SCREEN_HEIGHT - 10, 50, 10)
            self.walls.add(wall_top)
            self.walls.add(wall_bottom)
            self.all_sprites.add(wall_top)
            self.all_sprites.add(wall_bottom)
            wall_left = Wall(100, 200, 10, 150)
            wall_right = Wall(690, 200, 10, 150)
            self.walls.add(wall_left, wall_right)
            self.all_sprites.add(wall_left, wall_right)
            

        # Left and right walls
        for y in range(10, SCREEN_HEIGHT - 10, 50):
            wall_left = Wall(0, y, 10, 50)
            wall_right = Wall(SCREEN_WIDTH - 10, y, 10, 50)
            self.walls.add(wall_left)
            self.walls.add(wall_right)
            self.all_sprites.add(wall_left)
            self.all_sprites.add(wall_right)

    def generate_coins(self):
        for _ in range(10):
            coin = Coin()
            self.coins.add(coin)
            self.all_sprites.add(coin)
            
    def generate_obstacles(self):
        # Add your obstacle generation logic here
        # Example: Adding a single obstacle in the middle of the screen
        obstacle = Wall(SCREEN_WIDTH // 2 - 25, SCREEN_HEIGHT // 2 - 25, 50, 50)
        self.walls.add(obstacle)
        self.all_sprites.add(obstacle)

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            self.all_sprites.update()

            # Check collision between player and enemies
            if pygame.sprite.spritecollide(self.player, self.enemies, False):
                self.player.decrease_health(10)
                if self.player.health <= 0:
                    running = False
                    print("Game Over! Your score:", self.player.score)

            # Check collision between player and coins
            if pygame.sprite.spritecollide(self.player, self.coins, True):
                self.player.increase_score()
                if self.player.score >= 100:
                    running = False
                    print("Congratulations! You have won the game with a score of", self.player.score)

            self.screen.fill(WHITE)
            self.all_sprites.draw(self.screen)

            # Display score and health
            score_text = f"Score: {self.player.score}"
            score_surface = pygame.font.SysFont(None, 30).render(score_text, True, BLACK)
            self.screen.blit(score_surface, (10, 10))

            health_text = f"Health: {self.player.health}"
            health_surface = pygame.font.SysFont(None, 30).render(health_text, True, BLACK)
            self.screen.blit(health_surface, (10, 40))

            pygame.display.flip()
            self.clock.tick(60)

        pygame.quit()

game = Game()
game.run()

Game Over! Your score: 60
